Dhruv Kamboj 102303645 3C45 Assignment 6

In [1]:
#Q1
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

np.random.seed(42)

X1 = np.random.rand(100, 1)

X2 = X1 + 0.05 * np.random.randn(100, 1)
X3 = X1 + 0.1 * np.random.randn(100, 1)
X4 = X1 + 0.02 * np.random.randn(100, 1)
X5 = X1 + 0.03 * np.random.randn(100, 1)
X6 = X1 + 0.06 * np.random.randn(100, 1)
X7 = X1 + 0.07 * np.random.randn(100, 1)

X = np.hstack([X1, X2, X3, X4, X5, X6, X7])

y = 3 * X1 + 2 * X2 + 0.5 * X3 + 0.7 * X4 + 0.1 * X5 + 1.2 * X6 + 0.9 * X7 + np.random.randn(100, 1)

def ridge_regression_gd(X, y, learning_rate, lambda_reg, num_iterations=1000):
    m, n = X.shape
    X = np.c_[np.ones((m, 1)), X] 
    theta = np.zeros((n + 1, 1)) 

    for i in range(num_iterations):
        predictions = X.dot(theta)
        errors = predictions - y
        gradient = (1/m) * X.T.dot(errors) + (lambda_reg/m) * np.r_[[[0]], theta[1:]]  
        theta -= learning_rate * gradient

    return theta

def evaluate_model(X, y, learning_rates, lambdas):
    best_r2 = -np.inf
    best_params = None
    best_theta = None

    for lr in learning_rates:
        for lambda_reg in lambdas:
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            theta = ridge_regression_gd(X_train, y_train, lr, lambda_reg)

            X_test_ = np.c_[np.ones((X_test.shape[0], 1)), X_test]  
            y_pred = X_test_.dot(theta)

            r2 = r2_score(y_test, y_pred)
            if r2 > best_r2:
                best_r2 = r2
                best_params = (lr, lambda_reg)
                best_theta = theta

            print(f"Learning rate: {lr}, Lambda: {lambda_reg}, R2: {r2:.4f}")

    print(f"Best R2: {best_r2:.4f} with Learning rate: {best_params[0]} and Lambda: {best_params[1]}")
    return best_params, best_theta

learning_rates = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05]
lambdas = [1e-15, 1e-10, 1e-5, 1e-3, 0, 1, 10, 20]

best_params, best_theta = evaluate_model(X, y, learning_rates, lambdas)

Learning rate: 0.0001, Lambda: 1e-15, R2: -1.1193
Learning rate: 0.0001, Lambda: 1e-10, R2: -1.1193
Learning rate: 0.0001, Lambda: 1e-05, R2: -1.1193
Learning rate: 0.0001, Lambda: 0.001, R2: -1.1193
Learning rate: 0.0001, Lambda: 0, R2: -1.1193
Learning rate: 0.0001, Lambda: 1, R2: -1.1199
Learning rate: 0.0001, Lambda: 10, R2: -1.1253
Learning rate: 0.0001, Lambda: 20, R2: -1.1313
Learning rate: 0.0005, Lambda: 1e-15, R2: 0.6527
Learning rate: 0.0005, Lambda: 1e-10, R2: 0.6527
Learning rate: 0.0005, Lambda: 1e-05, R2: 0.6527
Learning rate: 0.0005, Lambda: 0.001, R2: 0.6527
Learning rate: 0.0005, Lambda: 0, R2: 0.6527
Learning rate: 0.0005, Lambda: 1, R2: 0.6502
Learning rate: 0.0005, Lambda: 10, R2: 0.6276
Learning rate: 0.0005, Lambda: 20, R2: 0.6021
Learning rate: 0.001, Lambda: 1e-15, R2: 0.8400
Learning rate: 0.001, Lambda: 1e-10, R2: 0.8400
Learning rate: 0.001, Lambda: 1e-05, R2: 0.8400
Learning rate: 0.001, Lambda: 0.001, R2: 0.8400
Learning rate: 0.001, Lambda: 0, R2: 0.8400


In [2]:
#Q2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
hitters = pd.read_csv('Hitters.csv')
hitters.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [4]:
#(a)
hitters.dropna(inplace=True)

In [5]:
hitters = pd.get_dummies(hitters, drop_first=True)

In [6]:
#(b)
X = hitters.drop("Salary", axis=1)
y = hitters["Salary"]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [9]:
#(c)
lin_reg = LinearRegression()
lin_reg.fit(X_train_scaled, y_train)
y_pred_lin = lin_reg.predict(X_test_scaled)


In [10]:
ridge_reg = Ridge(alpha=0.5748)
ridge_reg.fit(X_train_scaled, y_train)
y_pred_ridge = ridge_reg.predict(X_test_scaled)


In [11]:
lasso_reg = Lasso(alpha=0.5748)
lasso_reg.fit(X_train_scaled, y_train)
y_pred_lasso = lasso_reg.predict(X_test_scaled)


c:\Users\dhruv\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.100e+04, tolerance: 3.414e+03
  model = cd_fast.enet_coordinate_descent(


In [12]:
#(d)
models = {
    "Linear Regression": (y_pred_lin, lin_reg),
    "Ridge Regression": (y_pred_ridge, ridge_reg),
    "Lasso Regression": (y_pred_lasso, lasso_reg)
}

for name, (y_pred, model) in models.items():
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"\n{name}")
    print(f"Mean Squared Error: {mse:.2f}")
    print(f"R² Score: {r2:.4f}")



Linear Regression
Mean Squared Error: 150406.58
R² Score: 0.3806

Ridge Regression
Mean Squared Error: 145230.19
R² Score: 0.4019

Lasso Regression
Mean Squared Error: 146918.81
R² Score: 0.3950


In [13]:
#Q3
url = "https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv"
boston = pd.read_csv(url)
X = boston.drop("medv", axis=1)
y = boston["medv"]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [15]:
from sklearn.linear_model import RidgeCV, LassoCV
alphas = np.logspace(-3, 3, 50)

ridge_cv = RidgeCV(alphas=alphas, scoring='neg_mean_squared_error', cv=5)
ridge_cv.fit(X_train_scaled, y_train)

y_pred_ridge = ridge_cv.predict(X_test_scaled)


In [16]:
lasso_cv = LassoCV(alphas=alphas, cv=5, max_iter=10000)
lasso_cv.fit(X_train_scaled, y_train)

y_pred_lasso = lasso_cv.predict(X_test_scaled)


In [17]:
# Ridge performance
ridge_mse = mean_squared_error(y_test, y_pred_ridge)
ridge_r2 = r2_score(y_test, y_pred_ridge)

# Lasso performance
lasso_mse = mean_squared_error(y_test, y_pred_lasso)
lasso_r2 = r2_score(y_test, y_pred_lasso)

print(f"Best Alpha (Ridge): {ridge_cv.alpha_:.4f}")
print(f"Ridge - MSE: {ridge_mse:.4f}, R²: {ridge_r2:.4f}")

print(f"\nBest Alpha (Lasso): {lasso_cv.alpha_:.4f}")
print(f"Lasso - MSE: {lasso_mse:.4f}, R²: {lasso_r2:.4f}")


Best Alpha (Ridge): 14.5635
Ridge - MSE: 21.9287, R²: 0.7057

Best Alpha (Lasso): 0.0010
Lasso - MSE: 21.5215, R²: 0.7112


In [20]:
#Q4
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score, classification_report
iris = load_iris()
X = iris.data 
y = iris.target 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def compute_cost(X, y, theta):
    m = len(y)
    h = sigmoid(np.dot(X, theta)) 
    cost = (1/m) * np.sum(-y * np.log(h) - (1 - y) * np.log(1 - h))
    return cost

def gradient_descent(X, y, theta, learning_rate=0.01, iterations=1000):
    m = len(y)
    cost_history = []
    
    for _ in range(iterations):
        h = sigmoid(np.dot(X, theta))
        gradient = (1/m) * np.dot(X.T, (h - y))
        theta -= learning_rate * gradient  
        cost_history.append(compute_cost(X, y, theta))
    
    return theta, cost_history

class OneVsRestLogisticRegression:
    def __init__(self, num_classes):
        self.num_classes = num_classes
        self.models = []
    
    def fit(self, X, y):
        m, n = X.shape
    
        X = np.c_[np.ones(m), X]
        
        for class_label in range(self.num_classes):
            y_binary = (y == class_label).astype(int)
            
            theta_initial = np.zeros(X.shape[1])
            
            theta_optimal, _ = gradient_descent(X, y_binary, theta_initial)
            
            self.models.append(theta_optimal)
    
    def predict(self, X):
        m = X.shape[0]
        X = np.c_[np.ones(m), X]  
        
        probs = np.array([sigmoid(np.dot(X, theta)) for theta in self.models]).T
        predictions = np.argmax(probs, axis=1)
        
        return predictions

num_classes = len(np.unique(y_train))
model_ovr = OneVsRestLogisticRegression(num_classes)
model_ovr.fit(X_train_scaled, y_train)

y_pred = model_ovr.predict(X_test_scaled)

# Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8444

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      0.46      0.63        13
           2       0.65      1.00      0.79        13

    accuracy                           0.84        45
   macro avg       0.88      0.82      0.81        45
weighted avg       0.90      0.84      0.83        45

